# 03-sql.ipynb

1. 데이터베이스에서 사용 가능한 테이블과 스키마 가져오기.
1. 질문과 관련된 테이블을 LLM이 결정
1. 해당 테이블의 스키마 확인하기.
1. 질문과 스키마의 정보를 기반으로 쿼리를 생성.
1. LLM을 사용하여 흔히 발생하는 오류가 있는지 SQL 확인.
1. DB에서 SQL을 실행하고 결과를 확인.
1. DB에서 에러 발생시, 수정 후 다시 확인
1. DB 결과를 바탕으로 LLM이 답변 생성


In [2]:
# %pip install langgraph
# %pip install sqlalchemy
# %pip install psycopg2-binary  -- PostgreSQL 데이터베이스에 Python으로 연결할 때 필요한 드라이버

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
# https://docs.langchain.com/oss/python/langchain/sql-agent

from langchain_community.utilities import SQLDatabase
import os

DB_URI = os.getenv('DB_URI')

db = SQLDatabase.from_uri(DB_URI)

# DB 연결 잘 되었는지 DB 테이블 목록 보기
print(db.get_usable_table_names())
# DB table
print(db.get_table_info())
print(db.dialect)
print(db.run('SELECT * FROM sales LIMIT 5;'))

/Users/kimsiyeon/Desktop/gaida-2/.venv/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:25: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


['courses', 'customers', 'dt_demo', 'employees', 'lottery_infos', 'lotto_draws', 'members', 'sales', 'sample', 'students', 'students_courses', 'teams']

CREATE TABLE courses (
	id INTEGER GENERATED ALWAYS AS IDENTITY (INCREMENT BY 1 START WITH 1 MINVALUE 1 MAXVALUE 2147483647 CACHE 1 NO CYCLE), 
	name VARCHAR(50), 
	classroom VARCHAR(20), 
	CONSTRAINT courses_pkey PRIMARY KEY (id)
)

/*
3 rows from courses table:
id	name	classroom
1	MySQL 데이터베이스	A관 101호
2	PostgreSQL 고급	B관 203호
3	데이터 분석	A관 704호
*/


CREATE TABLE customers (
	customer_id VARCHAR(10) NOT NULL, 
	customer_name VARCHAR(50) NOT NULL, 
	customer_type VARCHAR(20) NOT NULL, 
	join_date DATE NOT NULL, 
	CONSTRAINT customers_pkey PRIMARY KEY (customer_id)
)

/*
3 rows from customers table:
customer_id	customer_name	customer_type	join_date
C001	김민수	VIP	2023-04-25
C002	이지은	개인	2023-10-09
C003	박서준	개인	2023-08-17
*/


CREATE TABLE dt_demo (
	id INTEGER GENERATED ALWAYS AS IDENTITY (INCREMENT BY 1 START WITH 1 MINVALUE 1 MAXVALUE 214748

In [5]:
# LLM 초기화
from langchain_openai import ChatOpenAI
model = ChatOpenAI(name = 'gpt-4.1-mini')

In [6]:
# Agent 용 Tool 만들기
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=model)

# toolkit.get_tools()는 Agent가 DB와 상호작용하는 데 필요한 tool들을 한번에 묶어서 제공하는 것 -> 여러가지가 있고 리스트 형태임 
# toolkit.get_tools()

# for tool in toolkit.get_tools():
#     print(tool.name, tool.description)

for tool in toolkit.get_tools():
    print(tool.name)

sql_db_query
sql_db_schema
sql_db_list_tables
sql_db_query_checker


In [7]:
# Agent 만들기
from langchain.agents import create_agent

# 어떤 DB 사용 하는지 알 수 있는 메서드
dialect = db.dialect

top_k = 5

system_prompt = f"""
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run,
then look at the results of the query and return the answer. Unless the user
specifies a specific number of examples they wish to obtain, always limit your
query to at most {top_k} results.

You can order the results by a relevant column to return the most interesting
examples in the database. Never query for all the columns from a specific table,
only ask for the relevant columns given the question.

You MUST double check your query before executing it. If you get an error while
executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the
database.

To start you should ALWAYS look at the tables in the database to see what you
can query. Do NOT skip this step.

Then you should query the schema of the most relevant tables.
"""



In [8]:
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver

# sql_db_query 하기 전에 humanintheloop -> 승인 받고 다시 돌아가서 멈춘 시점부터 그 다음 테스크를 해야해서 체크 포인트를 저장할 메모리 필요!
agent = create_agent(
    model, 
    toolkit.get_tools(),
    system_prompt=system_prompt,
    middleware =[
        HumanInTheLoopMiddleware(
            interrupt_on ={'sql_db_query': True},
            description_prefix ='Tool 실행 전에 승인을 기다림'
        )
    ],
    checkpointer = InMemorySaver() # 일시 정지 - 재실행에서 돌아갈 곳을 기억해야함!
    )

In [ ]:
from langgraph.types import Command 


question = '2월에 가장 많이 팔린 항목 3가지 알려줘'

config = {'configurable': {'thread_id': '12345'}}

for event in agent.stream(
    {'messages':[{'role':'user','content':question}]}, 
    stream_mode ='values', 
    config = config
):
    if "__interrupt__" in event: 
        print("INTERRUPTED:") 
        interrupt = event["__interrupt__"][0] 
        for request in interrupt.value["action_requests"]: 
            print(request["description"]) 
    elif "messages" in event:
        event["messages"][-1].pretty_print()
    else:
        pass

# 에이전트한테 질문을 보내고 실행시킨다

# 실행되는 매 단계마다 반복:
#     만약 "인터럽트"가 걸렸으면:
#         → "INTERRUPTED!" 출력
#         → 어떤 툴이 승인 대기 중인지 설명 출력
#         → (여기서 자연스럽게 멈춤)
    
#     만약 "메시지"가 있으면:
#         → 가장 마지막 메시지를 예쁘게 출력
    
#     그 외:
#         → 아무것도 안 함 (패스)

print('-----------------------------------------------------------------')

for step in agent.stream(
    Command(resume={"decisions": [{"type": "approve"}]}), 
    config,
    stream_mode="values",
):
    if "messages" in step:
        step["messages"][-1].pretty_print()
    elif "__interrupt__" in step:
        print("INTERRUPTED:")
        interrupt = step["__interrupt__"][0]
        for request in interrupt.value["action_requests"]:
            print(request["description"])
    else:
        pass

# 에이전트한테 "승인했어, 계속 해" 라고 보낸다

# 멈췄던 시점부터 이어서 매 단계마다 반복:
#     만약 "메시지"가 있으면:
#         → 가장 마지막 메시지를 예쁘게 출력
    
#     만약 또 "인터럽트"가 걸리면:
#         → "INTERRUPTED!" 출력
#         → 설명 출력
#         → (또 멈춤)
    
#     그 외:
#         → 아무것도 안 함 (패스)




================================ Human Message =================================

2월에 가장 많이 팔린 항목 3가지 알려줘
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_PwyEZNDNPTaQTJt1TcptWSil)
 Call ID: call_PwyEZNDNPTaQTJt1TcptWSil
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

courses, customers, dt_demo, employees, lottery_infos, lotto_draws, members, sales, sample, students, students_courses, teams
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_hbsM0Rgb9U1WD1GaO7VtSN9N)
 Call ID: call_hbsM0Rgb9U1WD1GaO7VtSN9N
  Args:
    table_names: sales
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE sales (
	id INTEGER NOT NULL, 
	order_date DATE NOT NULL, 
	customer_id VARCHAR(10) NOT NULL, 
	product_id VARCHAR(10) NOT NULL, 
	product_

In [11]:
# question = '전체 평균 매출액과, 가장 구매를 많이한 순서대로 손님의 이름 3명을 알려줘'

# for event in agent.stream(
#     {'messages':[{'role':'user','content':question}]}, stream_mode='values'
# ):
#     event['messages'][-1].pretty_print()

In [15]:
# agent.get_graph().print_ascii()

print(agent.get_graph().draw_mermaid())
print(agent.get_graph().nodes)

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	model(model)
	tools(tools)
	HumanInTheLoopMiddleware\2eafter_model(HumanInTheLoopMiddleware.after_model)
	__end__([<p>__end__</p>]):::last
	HumanInTheLoopMiddleware\2eafter_model -.-> __end__;
	HumanInTheLoopMiddleware\2eafter_model -.-> model;
	HumanInTheLoopMiddleware\2eafter_model -.-> tools;
	__start__ --> model;
	model --> HumanInTheLoopMiddleware\2eafter_model;
	tools -.-> model;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

{'__start__': Node(id='__start__', name='__start__', data=RunnableCallable(tags=None, recurse=True, explode_args=False, func_accepts={}), metadata=None), 'model': Node(id='model', name='model', data=model_node(tags=None, recurse=True, explode_args=False, func_accepts={'runtime': ('N/A', <class 'inspect._empty'>)}), metadata=None), 'tools': Node(id='tools', name='tools', data=tools(tags=None, recurse=True